In [ ]:
import numpy as np
from tqdm.notebook import tqdm_notebook
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

In [ ]:
images_resized = np.load('images_resized.npy')
depths_resized = np.load('depths_resized.npy')

In [ ]:
# Prepare data
X = images_resized.reshape(images_resized.shape[0], -1)
y_mean = depths_resized.reshape(depths_resized.shape[0], -1).mean(axis=1)

X_use, X_test, y_use, y_test = train_test_split(X, y_mean, test_size=0.2, random_state=42)

X_use = np.load("x_use.npy", allow_pickle=True)
X_test = np.load("x_test.npy", allow_pickle=True)
y_use = np.load("y_use.npy", allow_pickle=True)
y_test = np.load("y_test.npy", allow_pickle=True)

# Convert to tensors
X_use_tensor = torch.FloatTensor(X_use)
y_use_tensor = torch.FloatTensor(y_use).unsqueeze(1)
X_test_tensor = torch.FloatTensor(X_test)


In [ ]:
results = []
input_size = X_use.shape[1]

small_architectures = [
    [4],
    [8],
    [4, 4],
    [8, 8],
    [4, 4, 4],
    [8, 8, 8],
    [4, 4, 4, 4],
    [8, 8, 8, 8]]

medium_architectures = [
    [16],
    [32],
    [16, 16],
    [32, 32],
    [16, 16, 16],
    [32, 32, 32],
    [16, 16, 16, 16],
    [32, 32, 32, 32]]

large_architectures = [
    [256],
    [512],
    [1024]]

dropout = 0.2
lr = 0.001
weight_decays = [0]
batch_size = 64

iterations = 10
epochs = 10

In [ ]:
# MLP class
class MLP(nn.Module):
    def __init__(self, input_size, hidden_sizes, dropout=0.2):
        super(MLP, self).__init__()
        layers = []
        prev_size = input_size
        for hidden_size in hidden_sizes:
            layers.append(nn.Linear(prev_size, hidden_size))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout))
            prev_size = hidden_size
        layers.append(nn.Linear(prev_size, 1))
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x)

# Search function for MLP
def run_mlp_search(hidden_architectures, weight_decays, lr, batch_size, iterations, epochs, dropout=0.2):
    """
    Run architecture search for MLP regression models.
    """
    results = []
    input_size = X_use.shape[1]
    
    for hidden_sizes in tqdm_notebook(hidden_architectures, desc="Architecture"):
        for wd in tqdm_notebook(weight_decays, desc="Weight Decay", leave=False):
            
            iter_rmses = []
            iter_losses = []
            
            for iteration in tqdm_notebook(range(iterations), desc="Iteration", leave=False):
                # New train/val split each iteration
                X_train, X_val, y_train, y_val = train_test_split(
                    X_use, y_use, test_size=0.2, random_state=iteration
                )
                
                # Convert to tensors
                X_train_tensor = torch.FloatTensor(X_train)
                y_train_tensor = torch.FloatTensor(y_train).unsqueeze(1)
                X_val_tensor = torch.FloatTensor(X_val)
                
                # Create DataLoader
                train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
                train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
                
                # Initialize model
                model = MLP(input_size, hidden_sizes, dropout=dropout)
                criterion = nn.MSELoss()
                optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
                
                # Track losses
                train_losses = []
                
                # Train
                for epoch in range(epochs):
                    model.train()
                    epoch_loss = 0
                    for batch_X, batch_y in train_loader:
                        optimizer.zero_grad()
                        outputs = model(batch_X)
                        loss = criterion(outputs, batch_y)
                        loss.backward()
                        optimizer.step()
                        epoch_loss += loss.item()
                    
                    # Calculate average loss for this epoch
                    avg_loss = epoch_loss / len(train_loader)
                    train_losses.append(avg_loss)
                
                # Evaluate on validation
                model.eval()
                with torch.no_grad():
                    y_val_pred = model(X_val_tensor).numpy().flatten()
                    y_val_pred = np.maximum(y_val_pred, 0)
                    val_rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
                
                iter_rmses.append(val_rmse)
                iter_losses.append(train_losses)
            
            loss_array = np.array(iter_losses)
            mean_losses = np.mean(loss_array, axis=0)
            std_losses = np.std(loss_array, axis=0)
            
            # Store mean and std across iterations
            results.append({
                'hidden': hidden_sizes,
                'wd': wd,
                'val_rmse_mean': np.mean(iter_rmses),
                'val_rmse_std': np.std(iter_rmses),
                'val_rmses': iter_rmses,
                'train_losses': iter_losses,
                'train_loss_mean': mean_losses,
                'train_loss_std': std_losses
            })
    
    return results

In [ ]:
# Plot architecture results
def plot_architecture_results(results, hidden_architectures, set_num):
    """
    Plot RMSE vs architecture with error bars showing ±1 standard deviation.
    """
    import matplotlib.pyplot as plt
    
    weight_decays_unique = sorted(list(set([r['wd'] for r in results])))
    
    # Extract RMSE values for each architecture
    mean_rmse = []
    std_rmse = []
    arch_labels = []
    
    for arch in hidden_architectures:
        matching = [r for r in results if r['hidden'] == arch]
        if matching:
            mean_rmse.append(matching[0]['val_rmse_mean'])
            std_rmse.append(matching[0]['val_rmse_std'])
            arch_labels.append(str(arch))
    
    mean_rmse = np.array(mean_rmse)
    std_rmse = np.array(std_rmse)
    x_indices = np.arange(len(arch_labels))
    
    # Create plot
    fig, ax = plt.subplots(figsize=(7, 4))
    ax.plot(x_indices, mean_rmse, marker='o', label='Mean RMSE', linewidth=2, markersize=8)
    # Add light shading for standard deviation
    ax.fill_between(x_indices, mean_rmse - std_rmse, mean_rmse + std_rmse, 
                     alpha=0.3, label='± 1 Std Dev')
    
    ax.set_xlabel("Architecture (Hidden Layers)")
    ax.set_ylabel("RMSE (meters)")
    ax.set_ylim([0.5, 2.8])
    ax.minorticks_on()
    ax.grid(True, which='major', alpha=0.5)
    ax.grid(True, which='minor', alpha=0.3, linestyle=':')
    ax.set_xticks(x_indices)
    ax.set_xticklabels(arch_labels, rotation=45, ha='right')
    ax.legend(loc='best')
    
    plt.title(f"MLP Validation RMSE vs Architecture Set {set_num}")
    fig.tight_layout()
    plt.show()
    
    # Find best architecture
    best_idx_flat = np.argmin(mean_rmse)
    best_arch = hidden_architectures[best_idx_flat]
    best_wd = weight_decays_unique[0]  # Assuming single weight decay
    
    print(f"Best: Architecture={best_arch}, WD={best_wd:.0e}, RMSE={mean_rmse[best_idx_flat]:.3f} ± {std_rmse[best_idx_flat]:.3f}")
    
    # Return indices for compatibility
    best_idx = (0, best_idx_flat)  # (wd_idx, arch_idx)
    return best_idx, weight_decays_unique

# Evaluate best model on test set
def evaluate_on_test(best_architecture, best_wd, X_use, y_use, X_test, y_test, lr=0.001, batch_size=64, epochs=12, dropout=0.2):
    """
    Train final model on full training set and evaluate on test set.
    """
    input_size = X_use.shape[1]
    
    # Prepare full training set
    train_dataset = TensorDataset(
        torch.FloatTensor(X_use), 
        torch.FloatTensor(y_use).unsqueeze(1)
    )
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    # Initialize model
    final_model = MLP(input_size, best_architecture, dropout=dropout)
    optimizer = optim.Adam(final_model.parameters(), lr=lr, weight_decay=best_wd)
    criterion = nn.MSELoss()
    
    # Track losses
    train_losses = []
    
    # Train
    for epoch in range(epochs):
        epoch_loss = 0
        final_model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = final_model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        
        # Calculate average loss for this epoch
        avg_loss = epoch_loss / len(train_loader)
        train_losses.append(avg_loss)
    
    # Final test evaluation
    final_model.eval()
    X_test_tensor = torch.FloatTensor(X_test)
    with torch.no_grad():
        y_pred = final_model(X_test_tensor).numpy().flatten()
        y_pred = np.maximum(y_pred, 0)
        test_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    
    return test_rmse, train_losses, y_pred

# Plot test results
def plot_test_results(y_test, y_pred, train_losses, test_rmse):
    """
    Plot training loss curve and prediction scatter plot.
    """
    import matplotlib.pyplot as plt
    
    # Plot training loss over epochs
    plt.figure(figsize=(10, 6))
    plt.plot(train_losses)
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title(f"Loss vs Epochs")
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # Plot prediction scatter plot (similar to regression_training)
    plt.figure(figsize=(10, 6))
    plt.scatter(y_test, y_pred, alpha=0.5, s=10)
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2, label='Perfect prediction')
    plt.xlabel('Actual Depth (meters)')
    plt.ylabel('Predicted Depth (meters)')
    plt.title(f'MLP Predictions vs Actual Depth\nTest RMSE: {test_rmse:.3f} meters')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

In [ ]:
# Run architecture search
print("Running architecture search...")
small_results = run_mlp_search(
    hidden_architectures=small_architectures,
    weight_decays=weight_decays,
    lr=lr,
    batch_size=batch_size,
    iterations=iterations,
    epochs=epochs,
    dropout=dropout
)

np.save('mlp_results_small.npy', small_results)

In [ ]:
# Plot results
small_results=np.load('mlp_results_small.npy', allow_pickle=True)
best_idx, weight_decays_unique = plot_architecture_results(small_results, small_architectures, 1)

In [ ]:
# Evaluate on test set
print("Evaluating on test set...")
test_rmse, train_losses, y_pred = evaluate_on_test(
    best_architecture=medium_architectures[best_idx[1]],
    best_wd=weight_decays_unique[best_idx[0]],
    X_use=X_use,
    y_use=y_use,
    X_test=X_test,
    y_test=y_test,
    lr=lr,
    batch_size=batch_size,
    epochs=epochs,
    dropout=dropout
)

print(f"\nFinal Test RMSE: {test_rmse:.3f} meters")
# Plot training loss over epochs
plt.plot(train_losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title(f"Loss vs Epochs")
plt.show()

In [ ]:
# Run architecture search
print("Running architecture search...")
medium_results = run_mlp_search(
    hidden_architectures=medium_architectures,
    weight_decays=weight_decays,
    lr=lr,
    batch_size=batch_size,
    iterations=iterations,
    epochs=epochs,
    dropout=dropout
)

np.save('mlp_results_medium.npy', medium_results)

In [ ]:
# Plot results
medium_results=np.load('mlp_results_medium.npy', allow_pickle=True)
best_idx, weight_decays_unique = plot_architecture_results(medium_results, medium_architectures, 2)

In [ ]:
# Evaluate on test set
print("Evaluating on test set...")
test_rmse, train_losses, y_pred = evaluate_on_test(
    best_architecture=medium_architectures[best_idx[1]],
    best_wd=weight_decays_unique[best_idx[0]],
    X_use=X_use,
    y_use=y_use,
    X_test=X_test,
    y_test=y_test,
    lr=lr,
    batch_size=batch_size,
    epochs=epochs,
    dropout=dropout
)

print(f"\nFinal Test RMSE: {test_rmse:.3f} meters")
# Plot training loss over epochs
plt.plot(train_losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title(f"Loss vs Epochs")
plt.show()

In [ ]:
# Run architecture search
print("Running architecture search...")
large_results = run_mlp_search(
    hidden_architectures=large_architectures,
    weight_decays=weight_decays,
    lr=lr,
    batch_size=batch_size,
    iterations=iterations,
    epochs=epochs,
    dropout=dropout
)

np.save('mlp_results_large.npy', large_results)

In [ ]:
# Plot results
large_results=np.load('mlp_results_large.npy', allow_pickle=True)
best_idx, weight_decays_unique = plot_architecture_results(large_results, large_architectures, 3)

In [ ]:
# Evaluate on test set
print("Evaluating on test set...")
test_rmse, train_losses, y_pred = evaluate_on_test(
    best_architecture=large_architectures[best_idx[1]],
    best_wd=weight_decays_unique[best_idx[0]],
    X_use=X_use,
    y_use=y_use,
    X_test=X_test,
    y_test=y_test,
    lr=lr,
    batch_size=batch_size,
    epochs=epochs,
    dropout=dropout
)

print(f"\nFinal Test RMSE: {test_rmse:.4f} meters")
# Plot training loss over epochs
plt.plot(train_losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title(f"Loss vs Epochs")
plt.show()

In [ ]:
# Plot averaged training losses by architecture groups (small, medium, large)
import matplotlib.pyplot as plt

# Create single plot
plt.figure(figsize=(7, 4))

small_results_one = np.load('mlp_results_small.npy', allow_pickle=True)
medium_results_one = np.load('mlp_results_medium.npy', allow_pickle=True)
large_results_one = np.load('mlp_results_large.npy', allow_pickle=True)

# Plot small architectures (averaged)
for result in small_results_one:
    print(result)
    print(1)
    # Average across 10 iterations
    loss_array = np.array(result['train_losses'])
    mean_loss = loss_array[0]
    plt.plot(range(1, epochs+1), mean_loss, label=str(result['hidden']), linewidth=2)

# Plot medium architectures (averaged)
for result in medium_results_one:
    print(result)
    print(2)
    # Average across 10 iterations
    loss_array = np.array(result['train_losses'])
    mean_loss = loss_array[0]
    plt.plot(range(1, epochs+1), mean_loss, label=str(result['hidden']), linewidth=2)

# Plot large architectures (averaged)
for result in large_results_one:
    print(result)
    print(3)
    # Average across 10 iterations
    loss_array = np.array(result['train_losses'])
    mean_loss = loss_array[0]
    plt.plot(range(1, epochs+1), mean_loss, label=str(result['hidden']), linewidth=2)

plt.xlabel('Epoch')
plt.ylabel('MSE Loss')
plt.title('MLP MSE Training Loss Curves by Architecture')
plt.minorticks_on()
plt.xticks(range(1, epochs+1, 2))
plt.grid(True, which='major', alpha=0.5)
plt.grid(True, which='minor', alpha=0.3, linestyle=':')
plt.grid(True, alpha=0.3)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', title="Hidden Layer Architecture", ncol=2)
plt.tight_layout()
plt.show()